In [12]:
import tensorflow as tf
from tensorflow import keras

In [13]:
model = keras.models.load_model('./model_epoch_17_val_acc_0.9615.h5')

In [14]:
model

In [15]:
tf.saved_model.save(model, './model_hw_newhandpd_aug-ilum_resnet50')

In [4]:
!pip install tensorflow-serving-api==2.14.0 grpcio
!pip install grpcio
!pip install keras-image-helper

In [17]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [18]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [19]:
from keras_image_helper import create_preprocessor

In [20]:
preprocessor = create_preprocessor('resnet50', target_size=(224, 224))
url = 'https://github.com/andyathsid/parked-ml-backend/blob/dev-alt/hand-writing-prediction-service/sp1-P1.jpg?raw=true'
X = preprocessor.from_url(url)

In [21]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [36]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'model_hw_newhandpd_aug-ilum_resnet50'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_2'].CopyFrom(np_to_protobuf(X))

In [37]:
pb_request

model_spec {
  name: "model_hw_newhandpd_aug-ilum_resnet50"
  signature_name: "serving_default"
}
inputs {
  key: "input_2"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 224
      }
      dim {
        size: 224
      }
      dim {
        size: 3
      }
    }
    tensor_content: "\236\017\024C\'q\354B\327\243\314B\236\017\014C\'q\332B\327\243\276B\236\017\016C\'q\332B\327\243\302B\236\017\021C\'q\336B\327\243\314B\236\017\017C\'q\336B\327\243\312B\236\017\022C\'q\350B\327\243\310B\236\017\025C\'q\360B\327\243\326B\236\017\025C\'q\356B\327\243\332B\236\017\027C\'q\374B\327\243\342B\236\017\023C\'q\362B\327\243\324B\236\017\025C\'q\362B\327\243\336B\236\017\013C\'q\330B\327\243\310B\236\017\027C\'q\364B\327\243\340B\236\017\027C\'q\370B\327\243\336B\236\017\023C\'q\360B\327\243\326B\236\017\024C\'q\344B\327\243\322B\236\017\021C\'q\336B\327\243\324B\236\017\025C\'q\346B\327\243\334B\236\017\026C\'q\362B\327\243\330B\236\0

In [38]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [39]:
pb_response

outputs {
  key: "dense"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 2
      }
    }
    float_val: 0.06517317146062851
    float_val: 0.9348268508911133
  }
}
model_spec {
  name: "model_hw_newhandpd_aug-ilum_resnet50"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [29]:
preds = pb_response.outputs['dense'].float_val

In [33]:
preds[1]

0.9348268508911133